<a href="https://colab.research.google.com/github/harirm85/Hari_ML_workshop/blob/main/Home_Credit_Default_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount drive and connect to Kaggle

In [1]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Copy the Jason file and set security
!mkdir -p ~/.kaggle
!cp '/content/drive/MyDrive/Kaggle/kaggle.json' ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# # Upload Jason file from system manually 
# from google.colab import files
# files.upload()
# # Copy the Jason file and set security
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [3]:
#Download data
!kaggle competitions download -c home-credit-default-risk

100% 688M/688M [00:08<00:00, 145MB/s]
100% 688M/688M [00:08<00:00, 84.5MB/s]


In [4]:
import zipfile

# Specify the path to the downloaded zip file
zip_path = "/content/home-credit-default-risk.zip"

# Specify the directory to extract the files to
extract_path = "/content/home-credit-default-risk"

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
# import shutil

# # Specify the path to the directory where you want to save the dataset
# save_path = "/content/drive/MyDrive/Kaggle/home-credit-default-risk"

# # Create a copy of the extracted dataset in the save path
# shutil.copytree(extract_path, save_path)

# Import libraries

In [16]:
#!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:,.2f}'.format


# Verify data

In [12]:
app_data = pd.read_csv('/content/home-credit-default-risk/application_train.csv')
print(app_data.columns)

Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=122)


In [31]:
dtypes= {'NAME_CONTRACT_TYPE': 'category' ,'CODE_GENDER': 'category','FLAG_OWN_CAR': 'category','FLAG_OWN_REALTY': 'category'
  ,'NAME_TYPE_SUITE': 'category','NAME_INCOME_TYPE': 'category','NAME_EDUCATION_TYPE': 'category','NAME_FAMILY_STATUS': 'category'
  ,'NAME_HOUSING_TYPE': 'category','FLAG_MOBIL': 'category','FLAG_EMP_PHONE': 'category','FLAG_WORK_PHONE': 'category'
  ,'FLAG_CONT_MOBILE': 'category','FLAG_PHONE': 'category','FLAG_EMAIL': 'category','OCCUPATION_TYPE': 'category'
  ,'CNT_FAM_MEMBERS': 'category','REGION_RATING_CLIENT': 'category','REGION_RATING_CLIENT_W_CITY': 'category'  
  ,'WEEKDAY_APPR_PROCESS_START': 'category','HOUR_APPR_PROCESS_START': 'category','REG_REGION_NOT_LIVE_REGION': 'category'
  ,'REG_REGION_NOT_WORK_REGION': 'category','LIVE_REGION_NOT_WORK_REGION': 'category','REG_CITY_NOT_LIVE_CITY': 'category'
  ,'REG_CITY_NOT_WORK_CITY': 'category','LIVE_CITY_NOT_WORK_CITY': 'category','ORGANIZATION_TYPE': 'category'
  ,'FONDKAPREMONT_MODE': 'category','HOUSETYPE_MODE': 'category','WALLSMATERIAL_MODE': 'category', 'EMERGENCYSTATE_MODE': 'category'
  ,'FLAG_DOCUMENT_2': 'category','FLAG_DOCUMENT_3': 'category','FLAG_DOCUMENT_4': 'category', 'FLAG_DOCUMENT_5': 'category'
  ,'FLAG_DOCUMENT_6': 'category','FLAG_DOCUMENT_7': 'category','FLAG_DOCUMENT_8': 'category', 'FLAG_DOCUMENT_9': 'category'
  ,'FLAG_DOCUMENT_10': 'category','FLAG_DOCUMENT_11': 'category','FLAG_DOCUMENT_12': 'category', 'FLAG_DOCUMENT_13': 'category'
  ,'FLAG_DOCUMENT_14': 'category','FLAG_DOCUMENT_15': 'category','FLAG_DOCUMENT_16': 'category', 'FLAG_DOCUMENT_17': 'category'
  ,'FLAG_DOCUMENT_18': 'category','FLAG_DOCUMENT_19': 'category','FLAG_DOCUMENT_20': 'category', 'FLAG_DOCUMENT_21': 'category'
 }

app_data = app_data.astype(dtypes)

In [32]:
app_data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 122 columns):
 #    Column                        Non-Null Count   Dtype   
---   ------                        --------------   -----   
 0    SK_ID_CURR                    307511 non-null  int64   
 1    TARGET                        307511 non-null  int64   
 2    NAME_CONTRACT_TYPE            307511 non-null  category
 3    CODE_GENDER                   307511 non-null  category
 4    FLAG_OWN_CAR                  307511 non-null  category
 5    FLAG_OWN_REALTY               307511 non-null  category
 6    CNT_CHILDREN                  307511 non-null  int64   
 7    AMT_INCOME_TOTAL              307511 non-null  float64 
 8    AMT_CREDIT                    307511 non-null  float64 
 9    AMT_ANNUITY                   307499 non-null  float64 
 10   AMT_GOODS_PRICE               307233 non-null  float64 
 11   NAME_TYPE_SUITE               306219 non-null  category
 12   NAME_INCOME_TY

#Shape Data

In [33]:
label_col = 'TARGET'

feature_columns = app_data.columns

feature_columns = feature_columns.drop (['SK_ID_CURR', 'TARGET']) 
print (feature_columns)

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
       'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=120)


In [34]:
X = app_data[feature_columns]
y = app_data[[label_col]]

print (X.shape)
print (y.shape)

(307511, 120)
(307511, 1)


In [41]:
import xgboost as xgb
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

bst = xgb.XGBClassifier(tree_method="hist",enable_categorical=True)
bst.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [46]:
#y_pred = bst.predict(X_test)
accuracy = bst.score(X_test, y_test)
print(accuracy)

0.9195811586426679


In [59]:
fi_df = pd.DataFrame()

fi_df['Col'] = feature_columns
fi_df['imp'] = bst.feature_importances_

fi_df.sort_values(by='imp',ascending=False,inplace=True)

fi_df.head(50)

,Col,imp
40,EXT_SOURCE_2,0.04
41,EXT_SOURCE_3,0.04
1,CODE_GENDER,0.03
95,FLAG_DOCUMENT_3,0.02
11,NAME_EDUCATION_TYPE,0.02
0,NAME_CONTRACT_TYPE,0.02
8,AMT_GOODS_PRICE,0.02
29,REGION_RATING_CLIENT_W_CITY,0.02
39,EXT_SOURCE_1,0.02
6,AMT_CREDIT,0.01
